In [ ]:
pip install tensorflow matplotlib numpy opencv-python


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
# Set paths to your dataset
train_dir = '/content/drive/MyDrive/Artificial Intelligence part_1/Train'
test_dir = '/content/drive/MyDrive/Artificial Intelligence part_1/Test'

# Image data generators for training and testing
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20,
                                    width_shift_range=0.2, height_shift_range=0.2,
                                    shear_range=0.2, zoom_range=0.2,
                                    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load images from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Adjust based on your model input size
    batch_size=32,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')


In [ ]:
def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

model = create_model()


In [ ]:
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // train_generator.batch_size,
                    validation_data=test_generator,
                    validation_steps=test_generator.samples // test_generator.batch_size,
                    epochs=10)  # Adjust epochs as needed


In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()

plt.show()


In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy:.4f}')

# Predictions and confusion matrix
Y_pred = model.predict(test_generator)
y_pred = np.round(Y_pred).astype(int)

# Generate classification report and confusion matrix
print(classification_report(test_generator.classes, y_pred))
cm = confusion_matrix(test_generator.classes, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ['Real', 'Fake'])
plt.yticks(tick_marks, ['Real', 'Fake'])

thresh = cm.max() / 2.
for i, j in np.ndindex(cm.shape):
    plt.text(j, i, cm[i, j], horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy:.4f}')

# Predictions on test data
Y_pred = model.predict(test_generator)
y_pred = np.round(Y_pred).astype(int)


In [ ]:
# Generate classification report
print(classification_report(test_generator.classes, y_pred, target_names=['Real', 'Fake']))


In [ ]:
# Confusion matrix
cm = confusion_matrix(test_generator.classes, y_pred)

# Plot confusion matrix using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Real', 'Fake'], yticklabels=['Real', 'Fake'])
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Get indices of misclassified images
misclassified_indices = np.where(y_pred.flatten() != test_generator.classes)[0]

# Display misclassified images
plt.figure(figsize=(12, 12))
for i, idx in enumerate(misclassified_indices[:9]):  # Display first 9 misclassified images
    img = test_generator[idx][0][0]  # Get image from generator
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.title(f'True: {test_generator.classes[idx]}, Predicted: {y_pred[idx]}')
    plt.axis('off')
plt.show()


In [ ]:


from PIL import Image, ImageChops, ImageEnhance
import numpy as np
import matplotlib.pyplot as plt

def perform_ela(image_path, quality=95):
    """
    Perform Error Level Analysis (ELA) on an image.

    Args:
        image_path (str): Path to the input image.
        quality (int): Compression quality for ELA analysis.

    Returns:
        ela_image (PIL.Image.Image): ELA analysis result as an image.
    """
    # Load the original image
    original = Image.open(image_path).convert('RGB')

    # Save the image with the specified quality
    temp_path = 'temp_compressed.jpg'
    original.save(temp_path, 'JPEG', quality=quality)

    # Reopen the compressed image
    compressed = Image.open(temp_path)

    # Calculate the difference
    ela_image = ImageChops.difference(original, compressed)

    # Enhance the difference for better visualization
    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    scale = 255.0 / max_diff if max_diff != 0 else 1
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)

    return ela_image

def display_ela(image_path, ela_result):
    """
    Display the original and ELA images side by side.

    Args:
        image_path (str): Path to the input image.
        ela_result (PIL.Image.Image): ELA result as an image.
    """
    original = Image.open(image_path).convert('RGB')

    # Plot original and ELA result
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    axes[0].imshow(np.array(original))
    axes[0].set_title('Original Image')
    axes[0].axis('off')

    axes[1].imshow(np.array(ela_result))
    axes[1].set_title('ELA Result')
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()

# Example usage with your specific image path
image_path = '/content/drive/MyDrive/Artificial Intelligence part_1/images.jfif'  # Path to your image
ela_image = perform_ela(image_path)
display_ela(image_path, ela_image)
